In [19]:
import sys
sys.path.insert(0, r'../..')

In [20]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import pandas_profiling
import numpy as np

In [21]:
from lib.scoring import task_score
from lib.utils import convert_time, cur_time, cur_frame, start_table_to_dict, seconds_to_time

In [22]:
DATA_PATH = '../../data/train/events_data.csv'
STARTS_PATH = '../../data/train/real_start_train.csv'
EVENTS_TYPE = ['удар по воротам', 'угловой', 'замена', 'желтая карточка', 'гол']
SMALL_EVENTS_TYPE = ['замена', 'желтая карточка', 'гол']
EVENTS_COUNTS = [0.502, 0.223, 0.137, 0.09, 0.048]
TOTAL_TIME = 6000

In [23]:
starts = pd.read_csv(STARTS_PATH)
starts['file_name'] = starts['file_name'].apply(lambda name: name.split('_', 1)[1])
starts_dict = start_table_to_dict(starts)
starts

,Unnamed: 0,file_name,first_start,second_start
0,0,633012_5.mp4,930,4680
1,1,641579_3.mp4,53,3914
2,2,643734_5.mp4,2103,5840
3,3,631638_5.mp4,4200,7990
4,4,631646_5.mp4,4207,8027
5,5,631750_5.mp4,4203,7957
6,6,640600_3.mp4,1380,5100


In [24]:
data = pd.read_csv(DATA_PATH) 
data['event_time'] = data['event_time'].apply(convert_time)
data = data[data['file_name'].isin(starts['file_name'])]
data = data[data['event_type'].isin(SMALL_EVENTS_TYPE)]
data

,game_id,game_name,file_name,event_type,event_time,comments
155,1380660306,Локомотив – Спартак,643734_5.mp4,желтая карточка,1581,NaN
156,1380660306,Локомотив – Спартак,643734_5.mp4,желтая карточка,3102,NaN
157,1380660306,Локомотив – Спартак,643734_5.mp4,желтая карточка,3552,NaN
158,1380660306,Локомотив – Спартак,643734_5.mp4,желтая карточка,4698,NaN
159,1380660306,Локомотив – Спартак,643734_5.mp4,желтая карточка,4771,NaN
160,1380660306,Локомотив – Спартак,643734_5.mp4,желтая карточка,5577,NaN
161,1380660306,Локомотив – Спартак,643734_5.mp4,замена,4038,NaN
162,1380660306,Локомотив – Спартак,643734_5.mp4,замена,4093,NaN
163,1380660306,Локомотив – Спартак,643734_5.mp4,замена,4349,NaN
164,1380660306,Локомотив – Спартак,643734_5.mp4,замена,5156,NaN


In [25]:
data

,game_id,game_name,file_name,event_type,event_time,comments
155,1380660306,Локомотив – Спартак,643734_5.mp4,желтая карточка,1581,NaN
156,1380660306,Локомотив – Спартак,643734_5.mp4,желтая карточка,3102,NaN
157,1380660306,Локомотив – Спартак,643734_5.mp4,желтая карточка,3552,NaN
158,1380660306,Локомотив – Спартак,643734_5.mp4,желтая карточка,4698,NaN
159,1380660306,Локомотив – Спартак,643734_5.mp4,желтая карточка,4771,NaN
160,1380660306,Локомотив – Спартак,643734_5.mp4,желтая карточка,5577,NaN
161,1380660306,Локомотив – Спартак,643734_5.mp4,замена,4038,NaN
162,1380660306,Локомотив – Спартак,643734_5.mp4,замена,4093,NaN
163,1380660306,Локомотив – Спартак,643734_5.mp4,замена,4349,NaN
164,1380660306,Локомотив – Спартак,643734_5.mp4,замена,5156,NaN


In [26]:
data['event_time_video'] = [seconds_to_time(cur_frame(row.event_time, starts_dict[row.file_name]))
                            for _, row in data.iterrows()]
data['starts_first'] = [seconds_to_time(starts_dict[row.file_name][0])
                            for _, row in data.iterrows()]
data['starts_second'] = [seconds_to_time(starts_dict[row.file_name][1])
                            for _, row in data.iterrows()]

In [27]:
data = data.drop(['game_id', 'game_name', 'comments'], axis=1)

In [28]:
data = data[['file_name', 'event_time_video', 'event_time', 'event_type', 'starts_first', 'starts_second']]

In [29]:
data

,file_name,event_time_video,event_time,event_type,starts_first,starts_second
155,643734_5.mp4,61:24,1581,желтая карточка,35:03,97:20
156,643734_5.mp4,104:02,3102,желтая карточка,35:03,97:20
157,643734_5.mp4,111:32,3552,желтая карточка,35:03,97:20
158,643734_5.mp4,130:38,4698,желтая карточка,35:03,97:20
159,643734_5.mp4,131:51,4771,желтая карточка,35:03,97:20
160,643734_5.mp4,145:17,5577,желтая карточка,35:03,97:20
161,643734_5.mp4,119:38,4038,замена,35:03,97:20
162,643734_5.mp4,120:33,4093,замена,35:03,97:20
163,643734_5.mp4,124:49,4349,замена,35:03,97:20
164,643734_5.mp4,138:16,5156,замена,35:03,97:20


In [30]:
data.to_csv('../../output/temp/some_event_time_video.csv')